In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import tensorflow as tf
print("pandas:",pd.__version__,"numpy:",np.__version__,"tensorflow:",tf.__version__)

pandas: 1.1.1 numpy: 1.16.0 tensorflow: 1.14.0


In [2]:
def read_data(file_path):
    column_names = ['user-id', 'activity', 'timestamp', 'x-axis', 'y-axis', 'z-axis']
    data = pd.read_csv(file_path, header=None, names=column_names)
    return data

def feature_normalize(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

def windows(data, size):
    start = 0
    while start < data.count():
        yield start, start + size
        start += int(size / 2)

def segment_signal(data, window_size=90):
    segments = np.empty((0, window_size, 3))
    labels = np.empty((0))
    print(len(data['timestamp']))
    for (start, end) in windows(data['timestamp'], window_size):
        print((start, end))
        x = data["x-axis"][start:end]
        y = data["y-axis"][start:end]
        z = data["z-axis"][start:end]
        if (len(data['timestamp'][start:end]) == window_size):
            segments = np.vstack([segments, np.dstack([x, y, z])])
            labels = np.append(labels, stats.mode(data["activity"][start:end])[0][0])
    return segments, labels

def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x, W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x, kernel_size, num_channels, depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights), biases))

def apply_max_pool(x, kernel_size, stride_size):
    return tf.nn.max_pool2d(x, ksize=[1, 1, kernel_size, 1], strides=[1, 1, stride_size, 1], padding='VALID')


In [3]:
dataset = read_data('C:/Users/TUL/Desktop/CNN/WISDM_ar_v1.1/WISDM_ar_v1.1_raws.txt')
dataset['x-axis'] = feature_normalize(dataset['x-axis'])
dataset['y-axis'] = feature_normalize(dataset['y-axis'])
dataset['z-axis'] = feature_normalize(dataset['z-axis'])

In [4]:
segments, labels = segment_signal(dataset)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments), 1, 90, 3)
train_test_split = np.random.rand(len(reshaped_segments)) < 0.70
train_x = reshaped_segments[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_segments[~train_test_split]
test_y = labels[~train_test_split]

1098204
(0, 90)
(45, 135)
(90, 180)
(135, 225)
(180, 270)
(225, 315)
(270, 360)
(315, 405)
(360, 450)
(405, 495)
(450, 540)
(495, 585)
(540, 630)
(585, 675)
(630, 720)
(675, 765)
(720, 810)
(765, 855)
(810, 900)
(855, 945)
(900, 990)
(945, 1035)
(990, 1080)
(1035, 1125)
(1080, 1170)
(1125, 1215)
(1170, 1260)
(1215, 1305)
(1260, 1350)
(1305, 1395)
(1350, 1440)
(1395, 1485)
(1440, 1530)
(1485, 1575)
(1530, 1620)
(1575, 1665)
(1620, 1710)
(1665, 1755)
(1710, 1800)
(1755, 1845)
(1800, 1890)
(1845, 1935)
(1890, 1980)
(1935, 2025)
(1980, 2070)
(2025, 2115)
(2070, 2160)
(2115, 2205)
(2160, 2250)
(2205, 2295)
(2250, 2340)
(2295, 2385)
(2340, 2430)
(2385, 2475)
(2430, 2520)
(2475, 2565)
(2520, 2610)
(2565, 2655)
(2610, 2700)
(2655, 2745)
(2700, 2790)
(2745, 2835)
(2790, 2880)
(2835, 2925)
(2880, 2970)
(2925, 3015)
(2970, 3060)
(3015, 3105)
(3060, 3150)
(3105, 3195)
(3150, 3240)
(3195, 3285)
(3240, 3330)
(3285, 3375)
(3330, 3420)
(3375, 3465)
(3420, 3510)
(3465, 3555)
(3510, 3600)
(3555, 3645)
(

(26865, 26955)
(26910, 27000)
(26955, 27045)
(27000, 27090)
(27045, 27135)
(27090, 27180)
(27135, 27225)
(27180, 27270)
(27225, 27315)
(27270, 27360)
(27315, 27405)
(27360, 27450)
(27405, 27495)
(27450, 27540)
(27495, 27585)
(27540, 27630)
(27585, 27675)
(27630, 27720)
(27675, 27765)
(27720, 27810)
(27765, 27855)
(27810, 27900)
(27855, 27945)
(27900, 27990)
(27945, 28035)
(27990, 28080)
(28035, 28125)
(28080, 28170)
(28125, 28215)
(28170, 28260)
(28215, 28305)
(28260, 28350)
(28305, 28395)
(28350, 28440)
(28395, 28485)
(28440, 28530)
(28485, 28575)
(28530, 28620)
(28575, 28665)
(28620, 28710)
(28665, 28755)
(28710, 28800)
(28755, 28845)
(28800, 28890)
(28845, 28935)
(28890, 28980)
(28935, 29025)
(28980, 29070)
(29025, 29115)
(29070, 29160)
(29115, 29205)
(29160, 29250)
(29205, 29295)
(29250, 29340)
(29295, 29385)
(29340, 29430)
(29385, 29475)
(29430, 29520)
(29475, 29565)
(29520, 29610)
(29565, 29655)
(29610, 29700)
(29655, 29745)
(29700, 29790)
(29745, 29835)
(29790, 29880)
(29835, 29

(52245, 52335)
(52290, 52380)
(52335, 52425)
(52380, 52470)
(52425, 52515)
(52470, 52560)
(52515, 52605)
(52560, 52650)
(52605, 52695)
(52650, 52740)
(52695, 52785)
(52740, 52830)
(52785, 52875)
(52830, 52920)
(52875, 52965)
(52920, 53010)
(52965, 53055)
(53010, 53100)
(53055, 53145)
(53100, 53190)
(53145, 53235)
(53190, 53280)
(53235, 53325)
(53280, 53370)
(53325, 53415)
(53370, 53460)
(53415, 53505)
(53460, 53550)
(53505, 53595)
(53550, 53640)
(53595, 53685)
(53640, 53730)
(53685, 53775)
(53730, 53820)
(53775, 53865)
(53820, 53910)
(53865, 53955)
(53910, 54000)
(53955, 54045)
(54000, 54090)
(54045, 54135)
(54090, 54180)
(54135, 54225)
(54180, 54270)
(54225, 54315)
(54270, 54360)
(54315, 54405)
(54360, 54450)
(54405, 54495)
(54450, 54540)
(54495, 54585)
(54540, 54630)
(54585, 54675)
(54630, 54720)
(54675, 54765)
(54720, 54810)
(54765, 54855)
(54810, 54900)
(54855, 54945)
(54900, 54990)
(54945, 55035)
(54990, 55080)
(55035, 55125)
(55080, 55170)
(55125, 55215)
(55170, 55260)
(55215, 55

(76950, 77040)
(76995, 77085)
(77040, 77130)
(77085, 77175)
(77130, 77220)
(77175, 77265)
(77220, 77310)
(77265, 77355)
(77310, 77400)
(77355, 77445)
(77400, 77490)
(77445, 77535)
(77490, 77580)
(77535, 77625)
(77580, 77670)
(77625, 77715)
(77670, 77760)
(77715, 77805)
(77760, 77850)
(77805, 77895)
(77850, 77940)
(77895, 77985)
(77940, 78030)
(77985, 78075)
(78030, 78120)
(78075, 78165)
(78120, 78210)
(78165, 78255)
(78210, 78300)
(78255, 78345)
(78300, 78390)
(78345, 78435)
(78390, 78480)
(78435, 78525)
(78480, 78570)
(78525, 78615)
(78570, 78660)
(78615, 78705)
(78660, 78750)
(78705, 78795)
(78750, 78840)
(78795, 78885)
(78840, 78930)
(78885, 78975)
(78930, 79020)
(78975, 79065)
(79020, 79110)
(79065, 79155)
(79110, 79200)
(79155, 79245)
(79200, 79290)
(79245, 79335)
(79290, 79380)
(79335, 79425)
(79380, 79470)
(79425, 79515)
(79470, 79560)
(79515, 79605)
(79560, 79650)
(79605, 79695)
(79650, 79740)
(79695, 79785)
(79740, 79830)
(79785, 79875)
(79830, 79920)
(79875, 79965)
(79920, 80

(102015, 102105)
(102060, 102150)
(102105, 102195)
(102150, 102240)
(102195, 102285)
(102240, 102330)
(102285, 102375)
(102330, 102420)
(102375, 102465)
(102420, 102510)
(102465, 102555)
(102510, 102600)
(102555, 102645)
(102600, 102690)
(102645, 102735)
(102690, 102780)
(102735, 102825)
(102780, 102870)
(102825, 102915)
(102870, 102960)
(102915, 103005)
(102960, 103050)
(103005, 103095)
(103050, 103140)
(103095, 103185)
(103140, 103230)
(103185, 103275)
(103230, 103320)
(103275, 103365)
(103320, 103410)
(103365, 103455)
(103410, 103500)
(103455, 103545)
(103500, 103590)
(103545, 103635)
(103590, 103680)
(103635, 103725)
(103680, 103770)
(103725, 103815)
(103770, 103860)
(103815, 103905)
(103860, 103950)
(103905, 103995)
(103950, 104040)
(103995, 104085)
(104040, 104130)
(104085, 104175)
(104130, 104220)
(104175, 104265)
(104220, 104310)
(104265, 104355)
(104310, 104400)
(104355, 104445)
(104400, 104490)
(104445, 104535)
(104490, 104580)
(104535, 104625)
(104580, 104670)
(104625, 10471

...

(1000800, 1000890)
(1000845, 1000935)
(1000890, 1000980)
(1000935, 1001025)
(1000980, 1001070)
(1001025, 1001115)
(1001070, 1001160)
(1001115, 1001205)
(1001160, 1001250)
(1001205, 1001295)
(1001250, 1001340)
(1001295, 1001385)
(1001340, 1001430)
(1001385, 1001475)
(1001430, 1001520)
(1001475, 1001565)
(1001520, 1001610)
(1001565, 1001655)
(1001610, 1001700)
(1001655, 1001745)
(1001700, 1001790)
(1001745, 1001835)
(1001790, 1001880)
(1001835, 1001925)
(1001880, 1001970)
(1001925, 1002015)
(1001970, 1002060)
(1002015, 1002105)
(1002060, 1002150)
(1002105, 1002195)
(1002150, 1002240)
(1002195, 1002285)
(1002240, 1002330)
(1002285, 1002375)
(1002330, 1002420)
(1002375, 1002465)
(1002420, 1002510)
(1002465, 1002555)
(1002510, 1002600)
(1002555, 1002645)
(1002600, 1002690)
(1002645, 1002735)
(1002690, 1002780)
(1002735, 1002825)
(1002780, 1002870)
(1002825, 1002915)
(1002870, 1002960)
(1002915, 1003005)
(1002960, 1003050)
(1003005, 1003095)
(1003050, 1003140)
(1003095, 1003185)
(1003140, 10

(1020330, 1020420)
(1020375, 1020465)
(1020420, 1020510)
(1020465, 1020555)
(1020510, 1020600)
(1020555, 1020645)
(1020600, 1020690)
(1020645, 1020735)
(1020690, 1020780)
(1020735, 1020825)
(1020780, 1020870)
(1020825, 1020915)
(1020870, 1020960)
(1020915, 1021005)
(1020960, 1021050)
(1021005, 1021095)
(1021050, 1021140)
(1021095, 1021185)
(1021140, 1021230)
(1021185, 1021275)
(1021230, 1021320)
(1021275, 1021365)
(1021320, 1021410)
(1021365, 1021455)
(1021410, 1021500)
(1021455, 1021545)
(1021500, 1021590)
(1021545, 1021635)
(1021590, 1021680)
(1021635, 1021725)
(1021680, 1021770)
(1021725, 1021815)
(1021770, 1021860)
(1021815, 1021905)
(1021860, 1021950)
(1021905, 1021995)
(1021950, 1022040)
(1021995, 1022085)
(1022040, 1022130)
(1022085, 1022175)
(1022130, 1022220)
(1022175, 1022265)
(1022220, 1022310)
(1022265, 1022355)
(1022310, 1022400)
(1022355, 1022445)
(1022400, 1022490)
(1022445, 1022535)
(1022490, 1022580)
(1022535, 1022625)
(1022580, 1022670)
(1022625, 1022715)
(1022670, 10

(1039905, 1039995)
(1039950, 1040040)
(1039995, 1040085)
(1040040, 1040130)
(1040085, 1040175)
(1040130, 1040220)
(1040175, 1040265)
(1040220, 1040310)
(1040265, 1040355)
(1040310, 1040400)
(1040355, 1040445)
(1040400, 1040490)
(1040445, 1040535)
(1040490, 1040580)
(1040535, 1040625)
(1040580, 1040670)
(1040625, 1040715)
(1040670, 1040760)
(1040715, 1040805)
(1040760, 1040850)
(1040805, 1040895)
(1040850, 1040940)
(1040895, 1040985)
(1040940, 1041030)
(1040985, 1041075)
(1041030, 1041120)
(1041075, 1041165)
(1041120, 1041210)
(1041165, 1041255)
(1041210, 1041300)
(1041255, 1041345)
(1041300, 1041390)
(1041345, 1041435)
(1041390, 1041480)
(1041435, 1041525)
(1041480, 1041570)
(1041525, 1041615)
(1041570, 1041660)
(1041615, 1041705)
(1041660, 1041750)
(1041705, 1041795)
(1041750, 1041840)
(1041795, 1041885)
(1041840, 1041930)
(1041885, 1041975)
(1041930, 1042020)
(1041975, 1042065)
(1042020, 1042110)
(1042065, 1042155)
(1042110, 1042200)
(1042155, 1042245)
(1042200, 1042290)
(1042245, 10

(1059390, 1059480)
(1059435, 1059525)
(1059480, 1059570)
(1059525, 1059615)
(1059570, 1059660)
(1059615, 1059705)
(1059660, 1059750)
(1059705, 1059795)
(1059750, 1059840)
(1059795, 1059885)
(1059840, 1059930)
(1059885, 1059975)
(1059930, 1060020)
(1059975, 1060065)
(1060020, 1060110)
(1060065, 1060155)
(1060110, 1060200)
(1060155, 1060245)
(1060200, 1060290)
(1060245, 1060335)
(1060290, 1060380)
(1060335, 1060425)
(1060380, 1060470)
(1060425, 1060515)
(1060470, 1060560)
(1060515, 1060605)
(1060560, 1060650)
(1060605, 1060695)
(1060650, 1060740)
(1060695, 1060785)
(1060740, 1060830)
(1060785, 1060875)
(1060830, 1060920)
(1060875, 1060965)
(1060920, 1061010)
(1060965, 1061055)
(1061010, 1061100)
(1061055, 1061145)
(1061100, 1061190)
(1061145, 1061235)
(1061190, 1061280)
(1061235, 1061325)
(1061280, 1061370)
(1061325, 1061415)
(1061370, 1061460)
(1061415, 1061505)
(1061460, 1061550)
(1061505, 1061595)
(1061550, 1061640)
(1061595, 1061685)
(1061640, 1061730)
(1061685, 1061775)
(1061730, 10

(1078875, 1078965)
(1078920, 1079010)
(1078965, 1079055)
(1079010, 1079100)
(1079055, 1079145)
(1079100, 1079190)
(1079145, 1079235)
(1079190, 1079280)
(1079235, 1079325)
(1079280, 1079370)
(1079325, 1079415)
(1079370, 1079460)
(1079415, 1079505)
(1079460, 1079550)
(1079505, 1079595)
(1079550, 1079640)
(1079595, 1079685)
(1079640, 1079730)
(1079685, 1079775)
(1079730, 1079820)
(1079775, 1079865)
(1079820, 1079910)
(1079865, 1079955)
(1079910, 1080000)
(1079955, 1080045)
(1080000, 1080090)
(1080045, 1080135)
(1080090, 1080180)
(1080135, 1080225)
(1080180, 1080270)
(1080225, 1080315)
(1080270, 1080360)
(1080315, 1080405)
(1080360, 1080450)
(1080405, 1080495)
(1080450, 1080540)
(1080495, 1080585)
(1080540, 1080630)
(1080585, 1080675)
(1080630, 1080720)
(1080675, 1080765)
(1080720, 1080810)
(1080765, 1080855)
(1080810, 1080900)
(1080855, 1080945)
(1080900, 1080990)
(1080945, 1081035)
(1080990, 1081080)
(1081035, 1081125)
(1081080, 1081170)
(1081125, 1081215)
(1081170, 1081260)
(1081215, 10

In [5]:
input_height = 1
input_width = 90
num_labels = 6
num_channels = 3

batch_size = 10
kernel_size = 60
depth = 60


num_hidden = 1000

learning_rate = 0.0001

training_epochs = 8

total_batchs = reshaped_segments.shape[0] // batch_size

In [29]:
X = tf.compat.v1.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.compat.v1.placeholder(tf.float32, shape=[None,num_labels])

c1 = apply_depthwise_conv(X,kernel_size,num_channels,depth)
p = apply_max_pool(c1,20,2)
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10)

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

loss = -tf.reduce_sum(Y * tf.math.log(y_))
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

cost_history = np.empty(shape=[1], dtype=float)

In [39]:
print(test_y.shape)

(7380, 6)


In [36]:
with tf.compat.v1.Session() as session:
    tf.compat.v1.global_variables_initializer().run()
    for epoch in range(1):
        for b in range(total_batchs):
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            cost_history = np.append(cost_history, c)
        print("Epoch {}: Training Loss = {}, Training Accuracy = {}".format(
            epoch, c, session.run(accuracy, feed_dict={X: train_x, Y: train_y})))
    y_p = tf.argmax(y_, 1)
    y_true = np.argmax(test_y, 1)
    final_acc, y_pred = session.run([accuracy, y_p], feed_dict={X: test_x, Y: test_y})
    print("Testing Accuracy: {}".format(final_acc))
    temp_y_true = np.unique(y_true)
    temp_y_pred = np.unique(y_pred)
    np.save("y_true", y_true)
    np.save("y_pred", y_pred)
    print("temp_y_true", temp_y_true)
    print("temp_y_pred", temp_y_pred)
    print("Precision", precision_score(y_true.tolist(), y_pred.tolist(), average='weighted'))
    print("Recall", recall_score(y_true, y_pred, average='weighted'))
    print("f1_score", f1_score(y_true, y_pred, average='weighted'))
    print("confusion_matrix")
    print(confusion_matrix(y_true, y_pred))

Epoch 0: Training Loss = nan, Training Accuracy = 0.09310932457447052
Testing Accuracy: 0.0875338762998581
temp_y_true [0 1 2 3 4 5]
temp_y_pred [0]
Precision 0.007662179331820418
Recall 0.08753387533875338
f1_score 0.014090925359789357
confusion_matrix
[[ 646    0    0    0    0    0]
 [2326    0    0    0    0    0]
 [ 397    0    0    0    0    0]
 [ 313    0    0    0    0    0]
 [ 805    0    0    0    0    0]
 [2893    0    0    0    0    0]]


C:\Users\TUL\anaconda3\envs\tensor\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
print(y_p.shape)

(?,)
